In [1]:
from glob import glob
from sklearn.model_selection import train_test_split
import lmdb
import numpy as np
from scipy import signal
from scipy.io import wavfile

In [2]:
char_dict = {'2':0, '3':1, '4':2, '5':3, '6':4, '7':5, '8':6, '9':7, 
         'A':8, 'B':9, 'C':10, 'D':11, 'E':12, 'F':13, 'G':14, 'H':15, 'J':16, 'K':17, 'N':18, 'P':19, 'Q':20,
         'R':21, 'S':22, 'T':23, 'U':24, 'V':25, 'X':26, 'Y':27, 'Z':28}

In [3]:
training_set = lmdb.open('/Users/chienan/job/asr/competition/lmdb/train',map_size=1099511627776)

In [4]:
testing_set = lmdb.open('/Users/chienan/job/asr/competition/lmdb/test',map_size=1073741824)

In [5]:
dataset = glob('/Users/chienan/job/asr/competition/wav/*')

In [7]:
train, val = train_test_split(dataset, train_size=0.9, random_state=5213)

In [8]:
with training_set.begin(write=True) as db:
    for ind, d in enumerate(train):
        sr, wav = wavfile.read(d)
        f, t, Sxx = signal.spectrogram(wav, sr)
        Sxx = Sxx[:50,:] # 低頻的才是重點，高頻多為雜訊
        db.put(key='input-%05d'.encode()%(ind+1),
               value=Sxx.astype(np.float64).tobytes())
        db.put(key='label-%05d'.encode()%(ind+1),
               value=d.split('/')[-1][:-4].encode())
    db.put(key='num-samples'.encode(),
           value=str(ind+1).encode())

In [9]:
with testing_set.begin(write=True) as db:
    for ind, d in enumerate(val):
        sr, wav = wavfile.read(d)
        f, t, Sxx = signal.spectrogram(wav, sr)
        Sxx = Sxx[:50,:] # 低頻的才是重點，高頻多為雜訊
        db.put(key='input-%05d'.encode()%(ind+1),
               value=Sxx.astype(np.float64).tobytes())
        db.put(key='label-%05d'.encode()%(ind+1),
               value=d.split('/')[-1][:-4].encode())
    db.put(key='num-samples'.encode(),
           value=str(ind+1).encode())